In [1]:
import os
from dotenv import load_dotenv

load_dotenv()
from qdrant_client import QdrantClient

if os.getenv("IS_DOCKER"):
    qdrant_host = "localhost"
else:
    qdrant_host = os.getenv("QDRANT_HOST")
    
qdrant_url = f"http://{qdrant_host}:{os.getenv('QDRANT_PORT')}"
client = QdrantClient(url=qdrant_url)

In [2]:
collection_name = os.getenv("QDRANT_COLLECTION")
limit = client.count(collection_name=collection_name, exact=True).count

In [3]:
offset = None

scroll_results, _ = client.scroll(
    collection_name=collection_name,
    limit=limit,
    offset=offset,
    with_payload=True,
    with_vectors=True,
)

print(f"🤖 Generating synthetic queries for {len(scroll_results)} docs...")

🤖 Generating synthetic queries for 117 docs...


In [4]:
import os # Ensure os is imported
import json
from langchain_community.chat_models import ChatDeepInfra
from langchain_core.prompts import ChatPromptTemplate

def call_local_llm(template_str, variables, model="openai/gpt-oss-20b"):
    # Ensure API Key is set
    if not os.getenv("DEEPINFRA_API_TOKEN"):
        raise ValueError("Please set your DEEPINFRA_API_TOKEN environment variable.")

    llm = ChatDeepInfra(
        name=model, 
        temperature=0, 
        deepinfra_api_token=os.getenv("DEEPINFRA_API_TOKEN")
    )
    
    # Create template from the raw string
    prompt = ChatPromptTemplate.from_template(template_str)
    
    # Pipe: Prompt -> LLM
    chain = prompt | llm
    
    # Invoke with the dictionary of variables
    response = chain.invoke(variables)
    
    # Fix: Extract the text string from the AIMessage object
    return response.content

def generate_query_agent(doc_text):
    # CHANGE: Explicit instructions for Indonesian context
    prompt_template = """
    You are an Indonesian user of a beauty clinic application (like ERHA or Dermies).
    Read the following document snippet:
    "{doc_text}"
    
    Task: Write a natural search query in **Indonesian (Bahasa Indonesia)** that you would type to find this exact information.
    
    Guidelines:
    - If the text mentions a price (Rp ...), generate a query like "Berapa harga..." or "Biaya treatment...".
    - If the text mentions a location, generate a query like "Lokasi klinik..." or "Dimana cabang...".
    - Use natural, casual Indonesian if appropriate, but keep it clear.
    
    Constraint: Output ONLY the query string in Indonesian. Do NOT use English. No quotes.
    """
    
    # Pass the template AND the variable dict
    return call_local_llm(prompt_template, {"doc_text": doc_text}).strip()

def relevance_judge_agent(query, doc_text):
    # CHANGE: Contextualized the judge to understand Indonesian relevance
    prompt_template = """
    You are a relevance judge for an Indonesian search engine.
    
    Query (Indonesian): "{query}"
    Document Snippet: "{doc_text}"
    
    Task: Does the document contain the answer to the query or is it highly relevant?
    Output: Return strictly valid JSON like this: {json_format}
    """

    example_json = '{"is_relevant": 1}'
    
    response_text = call_local_llm(
        prompt_template, 
        {
            "query": query, 
            "doc_text": doc_text,
            "json_format": example_json
        }
    )
    
    try:
        # Find the first { and last }
        start = response_text.find('{')
        end = response_text.rfind('}') + 1
        if start == -1 or end == 0: return 0
        
        json_str = response_text[start:end]
        data = json.loads(json_str)
        return data.get('is_relevant', 0)
    except Exception as e:
        return 0

/Users/michaeleco/Documents/Github/zenith-ai/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
import pandas as pd
import os
from langchain_ollama.embeddings import OllamaEmbeddings

# Note: Ensure you have the 'call_local_llm' and 'relevance_judge_agent' 
# functions defined from the previous steps.

def build_dataset_with_ollama(client, collection_name, scroll_results):
    ollama_embedding = OllamaEmbeddings(
        model=os.getenv("OLLAMA_EMBEDDING_MODEL") or "nomic-embed-text",
        temperature=0
    )
    
    dataset = []
    total = len(scroll_results)
    
    print(f"🚀 Starting generation pipeline on {total} documents...")

    for i, point in enumerate(scroll_results):
        print(f"   Processing {i+1}/{total}...", end="\r")
        
        doc_text = point.payload.get('full_text', '')
        # Skip empty/short docs to save time and token costs
        if len(doc_text) < 50: continue 

        # --- AGENT 1: GENERATOR (INDONESIAN MODIFICATION) ---
        
        doc_type = point.payload.get('type', 'Text')
        
        # We clarify the context for the LLM
        if doc_type == "Table":
            context_hint = "This text is a pricelist table (Daftar Harga)."
        else:
            context_hint = "This text is a description of a treatment or product."
        
        # MODIFIED PROMPT: Explicitly requests Bahasa Indonesia
        prompt_gen = """
        You are an Indonesian user of a beauty clinic application (like ERHA or Dermies).
        Context: {context_hint}
        Document Snippet: "{doc_text}"
        
        Task: Write a natural search query in **Bahasa Indonesia** that you would type to find this exact information.
        
        Guidelines:
        - If it lists a price, ask like: "Berapa harga [treatment]?" or "Biaya [treatment]...".
        - If it describes a skin problem, ask like: "Solusi [jerawat/flek]..." or "Cara mengatasi...".
        - If it mentions a location, ask like: "Lokasi klinik di..." or "Cabang terdekat...".
        - Use casual but clear Indonesian.
        
        Constraint: Output ONLY the query string in Indonesian. Do NOT use quotes.
        """
        
        # Safe call with variables
        synthetic_query = call_local_llm(
            prompt_gen, 
            {"context_hint": context_hint, "doc_text": doc_text[:1200]} # Truncate to avoid context limit
        ).strip()
        
        # Validation: If query is too short or accidentally English
        if len(synthetic_query) < 5: continue
            
        # --- RETRIEVAL (Mining Hard Negatives) ---
        q_vector = ollama_embedding.embed_query(synthetic_query)
        
        search_result = client.query_points(
            collection_name=collection_name,
            query=q_vector,
            limit=4 # 1 Positive + 3 Potential Negatives
        )
        
        # Collect candidates: The retrieved hits + the original source doc
        candidates = {}
        
        # 1. Add Original Source (The "Gold" Positive)
        candidates[point.id] = point
        
        # 2. Add Retrieved Hits (Potential Hard Negatives)
        for hit in search_result.points:
            candidates[hit.id] = hit
            
        # --- AGENT 2: JUDGE (Ensures Quality) ---
        for doc_id, candidate_point in candidates.items():
            cand_text = candidate_point.payload.get('full_text', '')
            
            # The Judge decides: Is this candidate actually relevant to the Indonesian query?
            # Note: Ensure relevance_judge_agent is the updated one that understands Indonesian.
            is_relevant = relevance_judge_agent(synthetic_query, cand_text)
            
            dataset.append({
                "query": synthetic_query,
                "doc_id": doc_id,
                "full_text": cand_text,
                "h1": candidate_point.payload.get('h1', ''), 
                "h2": candidate_point.payload.get('h2', ''),
                "qdrant_score": candidate_point.score if hasattr(candidate_point, 'score') else 0.0, 
                "label": is_relevant
            })
            
    print(f"\n✅ Finished! Generated {len(dataset)} training examples.")
    return pd.DataFrame(dataset)

# Usage
df_train = build_dataset_with_ollama(client, collection_name, scroll_results)
df_train.head(15)

🚀 Starting generation pipeline on 117 documents...
   Processing 117/117...
✅ Finished! Generated 513 training examples.


,query,doc_id,full_text,h1,h2,qdrant_score,label
0,Harga perawatan mata panda di ERHA Ultimate Br...,03d83c62-3d47-4746-8b2c-13b78eaee3d8,PROSEDUR\n\nCleansing - Treatment - Post treat...,724126666-ERHA-Ultimate-Pricelist-24.pdf,,0.631898,1
1,Harga perawatan mata panda di ERHA Ultimate Br...,05aa6050-6960-4715-a97e-0a85937ce505,ERHA ULTIMATE\n\nACNE CENTER\n\nPROGRAMS\n\nAc...,724126666-ERHA-Ultimate-Pricelist-24.pdf,,0.656354,0
2,Harga perawatan mata panda di ERHA Ultimate Br...,6b8ef39f-e9b0-4327-9313-b7773fcc18d9,BRIGHTENING CENTER\n\nSINGLE TREATMENT\n\nCrys...,724126666-ERHA-Ultimate-Pricelist-24.pdf,,0.646983,0
3,Harga perawatan mata panda di ERHA Ultimate Br...,17429cdb-1a3d-4345-885e-4e6e7e80db83,Kebiasaan Merokok\n\nRokok tidak hanya menyeba...,661627558-Katalog-Dermies-Max-by-Erha-Hiress.pdf,,0.641863,0
4,Perawatan jerawat punggung dan dada dengan tek...,04b23e8a-a1c2-4ef3-9a64-8031881a498a,SINGLE TREATMENT\n\n2) 20 menit ® Mulai dari R...,724126666-ERHA-Ultimate-Pricelist-24.pdf,,0.628178,1
5,Perawatan jerawat punggung dan dada dengan tek...,fd8aad2b-ba61-45d5-86d3-52ecb288f48a,"DERMIAS MAx\n\nApa bedanya Shaving, Waxing, IP...",661627558-Katalog-Dermies-Max-by-Erha-Hiress.pdf,,0.636846,1
6,Perawatan jerawat punggung dan dada dengan tek...,a9fd54d1-23eb-4c1a-8fa0-f33fcd6d0f99,D) 25 menit @) Rp325.000\n\nMANFAAT\n\nMencera...,724126666-ERHA-Ultimate-Pricelist-24.pdf,,0.630196,0
7,Perawatan jerawat punggung dan dada dengan tek...,28e15f29-81ce-4431-a00f-2c4be205c866,Apa Keunggulannya?\n\n|\n\nHempaskan rambut ha...,661627558-Katalog-Dermies-Max-by-Erha-Hiress.pdf,,0.616886,0
8,Program perawatan jerawat di ERHA Ultimate,05aa6050-6960-4715-a97e-0a85937ce505,ERHA ULTIMATE\n\nACNE CENTER\n\nPROGRAMS\n\nAc...,724126666-ERHA-Ultimate-Pricelist-24.pdf,,0.678199,1
9,Program perawatan jerawat di ERHA Ultimate,5034f916-a0ac-4db3-95fa-9a4d0728e8fc,Melasma\n\nJenis flek hitam yang lebih besar d...,724126666-ERHA-Ultimate-Pricelist-24.pdf,,0.620877,0


In [6]:
df_train.to_csv("../data/csv/rerank_train.csv", index=False)